# Experiment #9:  Introduction to Natural Language Processing
<b>Mohammed Abed Alkareem</b>
<b>1210708</b>

In [1]:
# !pip install -U datasets transformers[torch] evaluate

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   ---------------------------------------- 547.8/547.8 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   --- ------------------------------------ 2.1/25.1 MB 9.8 MB/s eta 0:00:03
   ------- -------------------------------- 4.7/25.1 MB 11.0 MB/s eta 0:00:02
   ----------- ---------------------------- 7.1/25.1 MB 11.2 MB/s eta 0:00:02
   --------------- ------------------------ 9.7/25.1 MB 11.2 MB/s eta 0:00:02
   ------------------- -------------------- 12.1/25.1 MB 11.3 MB/s eta 0:00:02
   ---------------------- ----------------- 14.2/25.1 MB 11.0 MB/s eta 0:00:01
   ------------------------ --------------- 15.5/25.1 MB 10.5 MB/s eta 0:00:01
   ------------------------- -------------- 16.3/25.1 MB 9.5 MB/s eta 0:00:01
   ------

In [2]:
from datasets import load_dataset

dataset = load_dataset('MrbBakh/Sentiment140')

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

## 1.2.1  Text Pre-processing

In [4]:
# !pip install nltk

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 6.7 MB/s eta 0:00:00


### tokenizing the text

In [5]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize(row):
    tokens = word_tokenize(row['text'])
    # to lowercase and remove punctuation
    tokens = [token.lower() for token in tokens if token.isalpha()]
    return {'tokens': tokens}

dataset = dataset.map(tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

### remove stop words

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(row):
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in row['tokens'] if token not in stop_words]
    return {'tokens': tokens}

dataset = dataset.map(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

### Task 1:  
UsePorterStemmerfrom NLTK to stem the tokens.

## 1.2.2  Word Embedding

In [8]:
# !pip install gensim

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/24.0 MB 8.4 MB/s eta 0:00:03
   ----- ---------------------------------- 3.1/24.0 MB 10.8 MB/s eta 0:00:02
   --------- ------------------------------ 5.8/24.0 MB 10.3 MB/s eta 0:00:02
   ------------- -------------------------- 7.9/24.0 MB 10.3 MB/s eta 0:00:02
   ---------------- ----------------------- 10.0/24.0 MB 10.5 MB/s eta 0:00:02
   ------------------- -------------------- 11.5/24.0 MB 10.0 MB/s eta 0:00:02
   ---------------------- ----------------- 13.4/24.0 MB 9.6 MB/s eta 0:00:02
   ------------------------- -------------- 15.2/24.0 MB 9.5 MB/s eta 0:00:01
   --------------------------- ------------ 16.8/24.0 MB 9.3 MB/s eta 0:00:01
   ------------------------------- -------- 18.6/24.0 MB 9.2 MB/s eta 0:00:01
   ---------------------------------- ----- 20.4/24.0 MB 9.1 MB/s eta 0

  You can safely remove it manually.
  You can safely remove it manually.


In [9]:
from gensim.models import Word2Vec

word_embedding = Word2Vec(dataset['train']['tokens'],
                           vector_size=100,
                           min_count=1,
                             window=5,
                               sg=1,
                                 hs=0,
                                   negative=10)

In [10]:
#save the model
word_embedding.save('w2v.model')

#load the model
word_embedding = Word2Vec.load('w2v.model')

## 1.2.3  Average Vector

In [11]:
def filter_tokens(example):
    return {
        'tokens': [token for token in example['tokens'] if token in word_embedding.wv]
        }
    
def mean_vector(example):
    return {
        'mean': word_embedding.wv[example['tokens']].mean(axis=0)
    }


dataset = dataset.map(filter_tokens)\
    .filter(lambda e: len(e['tokens']) > 0) \
    .map(mean_vector)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4961 [00:00<?, ? examples/s]

Map:   0%|          | 0/4964 [00:00<?, ? examples/s]

### Train the Naive Bayes classifier

In [12]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

X = np.array(dataset['train']['mean'])
y = np.array(dataset['train']['sentiment'])
clf = GaussianNB()
clf.fit(X, y)

GaussianNB()

#### Task 2:  
Compute the accuracy and the confusion matrix of the trained classifier onthe test dataset.

## 1.2.4  LSTM